<a href="https://colab.research.google.com/github/Pavun-KumarCH/Agentic-RAG-Systems/blob/main/BQP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
#@title requirements
%pip install --q langchain_groq langchain_core

In [28]:
#@title Libraries
import os, time, queue, threading
from typing import List, Dict
from datetime import datetime
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq

# Environment
from google.colab import userdata
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')

In [29]:
#@title Initializing Mistral:7B Model

llm = ChatGroq(
  model = "gemma2-9b-it",
  temperature = 0.4,
  max_tokens = None,
  timeout = None,
  max_retries = 2
)

In [32]:
#@title Prompt Template for Technical Support Bot
prompt_template = """Tech Support Conversation:
{conversation}
AI: You are a chatbot simulating a technical support representative, providing clear, concise guidance to users with technical issues.
"""
# PromptTemplate Instance
chat_prompt = PromptTemplate(
  template = prompt_template,
  input_variables = ["conversation"]
)

In [23]:
#@title Class Batch Processor
class BatchProcessor:
  def __init__(self, processor_id: int, batch_size: int, batch_interval: float):
    self.processor_id = processor_id
    self.batch_size = batch_size
    self.batch_interval = batch_interval
    self.request_queue = queue.Queue()
    self.lock = threading.Lock()
    self._stop_event = threading.Event()
    self.worker_thread = None

# Start Batching
  def start_batching(self):
    if self.worker_thread is None or not self.worker_thread.is_alive():
      self._stop_event.clear()
      self.worker_thread = threading.Thread(target = self.batch_worker)
      self.worker_thread.start()
      print(f"Batch processor {self.processor_id} started batching.")

# Add Request
  def add_request(self, request_data: Dict[str, str]):
    self.request_queue.put(request_data)
    print(f"{request_data}Request added to processor {self.processor_id}.")

# Process Batch
  def process_batch_with_langchain(self, batch: List[Dict[str, str]]):
    formatted_prompts = [
        chat_prompt.format(conversation = "\n".join(f"{item['role'].capitalize()}: {item['content']}" for item in [request]))
        for request in batch
    ]

    # Batch Method
    responses = llm.batch(formatted_prompts)
    for response in responses:
      print(f"Response from processor {self.processor_id}: {response}")
      print(response, "\n")

# Batch Worker
  def batch_worker(self):
    while not self._stop_event.is_set():
      batch = []
      start_time = time.time()
      while len(batch) < self.batch_size and (time.time() - start_time) < self.batch_interval:
        try:
          request_data = self.request_queue.get(timeout = self.batch_interval - (time.time() - start_time))
          batch.append(request_data)
          self.request_queue.task_done()
        except queue.Empty:
          break

      if batch:
        print(f"Processor {self.processor_id} - Processing batch of size {len(batch)}")
        with self.lock:
          self.process_batch_with_langchain(batch)
        print(f"Processor {self.processor_id} - Batch processor complete.\n")
      else:
        print(f"Processor {self.processor_id} - No requests  to process in the cycle.")


# Stop Batching
  def stop_batching(self):
    self._stop_event.set()
    if self.worker_thread:
      self.worker_thread.join()
      self.worker_thread = None
      print(f"Batch processor {self.processor_id} stopped batching.")

In [33]:
#@title Class Load Balancer -> round robin distribution
class LoadBalancer:
  def __init__(self, processors:List['BatchProcessor']):
    self.processors = processors
    self.current_processor_index = 0

  def distribute_request(self, request_data: Dict[str, str]):
    processor = self.processors[self.current_processor_index]
    processor.add_request(request_data)
    self.current_processor_index = (self.current_processor_index + 1) % len(self.processors)

# Chat Interactions as Incoming Requests
chat_interactions = [
    {"role": "user", "content": "Hi, I'm having trouble installing the software on my computer."},
    {"role": "user", "content": "My application keeps crashing. Any advice on fixing this?"},
    {"role": "user", "content": "How do I reset my account password?"},
    {"role": "user", "content": "The update process seems stuck. What should I do?"},
    {"role": "user", "content": "Is there a way to recover deleted files in the program?"},
    {"role": "user", "content": "How can I clear the cache to improve app performance?"},
    {"role": "user", "content": "I'm getting an error code when I try to launch the app."},
    {"role": "user", "content": "Can you help me troubleshoot connectivity issues with the server?"},
    {"role": "user", "content": "The software isn't recognizing my device. What can I do?"},
    {"role": "user", "content": "How do I configure the settings for optimal performance?"}
]

# Initializing two batch processors
batch_processor1 = BatchProcessor(processor_id = 1, batch_size = 2, batch_interval = 10)
batch_processor2 = BatchProcessor(processor_id = 2, batch_size = 2, batch_interval = 10)

# Initializing load balancer with processors
load_balancer = LoadBalancer(processors = [batch_processor1, batch_processor2])

# Starting batching for both processors
batch_processor1.start_batching()
batch_processor2.start_batching()

# Adding each meassge to the load balancer with a timestamp
for interaction in chat_interactions:
  interaction_with_timestamp = interaction.copy()
  interaction_with_timestamp["timestamp"] = datetime.now().isoformat()
  load_balancer.distribute_request(interaction_with_timestamp)
  time.sleep(1)

# Stopping batching for both processors
batch_processor1.stop_batching()
batch_processor2.stop_batching()

Batch processor 1 started batching.
Batch processor 2 started batching.
{'role': 'user', 'content': "Hi, I'm having trouble installing the software on my computer.", 'timestamp': '2024-11-11T16:57:14.104439'}Request added to processor 1.
{'role': 'user', 'content': 'My application keeps crashing. Any advice on fixing this?', 'timestamp': '2024-11-11T16:57:15.105619'}Request added to processor 2.
{'role': 'user', 'content': 'How do I reset my account password?', 'timestamp': '2024-11-11T16:57:16.106831'}Request added to processor 1.Processor 1 - Processing batch of size 2

Response from processor 1: content="AI:  I'm sorry to hear you're having trouble installing the software. I'm here to help!  \n\nTo get started, can you tell me:\n\n* **What is the name of the software you're trying to install?**\n* **What operating system are you using (Windows, Mac, etc.)?**\n* **What specific error message are you seeing, if any?**\n\n\nThe more information you can give me, the better I can assist 